# 📘 Lesson 1 — LangChain Fundamentals for Agentic Development

In [46]:
import os
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI

load_dotenv()

True

Before we can build an autonomous agent that can **reason** and **act**, we must first understand the basic tools used to build it.

LangChain does **not** work because of one magical `"agent"` function.
Its real power comes from a simple but strong idea:

> **Composability**

This lesson is fully dedicated to this core principle.


## 🔹 Core Philosophy: Composability and LCEL

LangChain is built around small components that can be **combined together**.
Each component does **one job**, and we connect them step by step.

This idea is inspired by pipelines:

* input goes in,
* transformations happen,
* output comes out.

LangChain formalizes this idea with the **LangChain Expression Language (LCEL)**.


### 🧱 The Three Core Components

In most LangChain applications, you will work with these three elements:

* **Prompt** :
  Defines *what* we ask the model.

* **Model** :
  The language model that generates the response.

* **Output Parser** :
  Transforms the raw model output into a usable format.

A very simple chain looks like this:

```
Prompt | Model | Output Parser
```

This is the **foundation of agentic systems**.
Even complex agents are built by composing simple chains like this.

### 🔹 Building Our First Chain with LCEL

Let’s now build our **first LangChain pipeline** to see composability in action.

The goal is simple:

> Generate a short and funny tagline for a company.

> ✍️ Step 1 — Define the Prompt Template

In [ ]:
promp_template = ChatPromptTemplate.from_template("Generate a short, funny tagline for a company that makes: {topic}")

This prompt contains a **variable** called `{topic}`.
It allows us to reuse the same prompt with different inputs.


> 🧠 Step 2 — Instantiate the Model

In [4]:
model =  ChatOpenAI(model="gpt-4o", temperature=0.0)


* We use `gpt-4o`
* `temperature=0` means:

  * more stable
  * more deterministic
  * good for reproducible behavior

> 🔄 Step 3 — Define the Output Parser

In [5]:
output_parser = StrOutputParser()

The model returns a complex object.
The `StrOutputParser` extracts **only the text**, as a simple string.

> 🔗 Step 4 — Compose the Chain with LCEL

Here, we define a processing pipeline.

In [7]:
chain = promp_template | model | output_parser

- The prompt formats the input data.

- The language model generates a response from that prompt.

- The output parser transforms the model response into a simple string.

Each component performs a single transformation.
LCEL makes these transformations explicit and ordered.

This way of composing logic is central to LangChain:

- behavior is built by composition, not by inheritance,

- complex systems emerge from simple, connected blocks.

> ▶️ Step 5 — Invoke the Chain

Here:

* we pass the input as a dictionary
* the chain runs from start to end
* we receive a clean string as output

In [9]:
topic_input = {"topic": "Agentic AI"}
result = chain.invoke(topic_input)
print(f"Input Topic: {topic_input['topic']}")
print(f"Generated Tagline: {result}")

Input Topic: Agentic AI
Generated Tagline: "Agentic AI: Because even your to-do list deserves a personal assistant with a sense of humor!"


At this stage, we did **not** build an agent yet.

But we learned something more important:

* LangChain is **modular**
* Every agent is built from **simple chains**
* LCEL makes data flow **explicit and readable**

This pattern:

```
Prompt | Model | Output Parser
```

will appear again and again when building agentic systems.

### Practical LCEL — Piping Components Together for Inputs and Outputs

In the previous section, the chain was simple:  
one input variable → one prompt → one output.

Real-world applications are rarely that simple.

Very often, a chain must work with **several pieces of information at the same time**.
For example:
- a name and a description,
- a question and some context,
- user input and retrieved data.

LCEL supports this kind of data flow through objects called **Runnables**.
Runnables allow us to **control how information moves and transforms** inside a chain.

In this section, we focus on two common Runnable patterns:
- `RunnablePassthrough`
- `RunnableParallel`

We start with the most basic one.

### Handling Multiple Inputs with RunnablePassthrough

The goal is simple:

> Take one input dictionary and make its values available to the prompt in a clean and explicit way.

We will build a small preparation step that passes data forward without modifying its meaning.

#### Step 1 — Define the Core Components

In [17]:
model = ChatOpenAI(model="gpt-4o", temperature=0.5)
promp_template = ChatPromptTemplate.from_template(
    "Write a short biography about {name}. "
    "Focus on the following contribution: {context}. "
    "Keep it light and readable."
)
output_parser = StrOutputParser()

The prompt expects a dictionary like:

```python
{"name": "...", "context": "..."}
```
In practice, input data can:

- come from another chain,

- contain extra fields,

- or be produced dynamically.

So we often need an explicit preparation step to control what the prompt receives.

#### Step 2 — Passing Data Forward with RunnablePassthrough Logic

The idea behind `RunnablePassthrough` is simple: take the input from the previous step and make it available downstream.

Here, we build a small mapping that:
- receives the full input dictionary,
- extracts the fields we need,
- produces a clean dictionary for the prompt.

In [13]:
setup_and_retrieval = {
    "name": lambda x: x["name"],
    "context": lambda x: x["context"]
}

This step does not generate new information.
It only **selects and forwards** existing values.

This pattern appears very often in agentic systems:
- passing user input,
- passing tool outputs,
- passing retrieved context.

#### Step 3 — Compose the Full Chain

The data flow is now explicit:

- the input dictionary enters the setup step,
- the setup step prepares the prompt variables,
- the prompt formats a message,
- the model generates text,
- the parser extracts a clean string.

In [20]:
chain = setup_and_retrieval | promp_template | model | output_parser

In [21]:
input_data = {
    "name": "Albert Einstein",
    "context": "the theory of relativity"
}

result = chain.invoke(input_data)

print("----- Biography Generation -----")
print(f"Input: {input_data}")
print("\nGenerated Biography:")
print(result)


----- Biography Generation -----
Input: {'name': 'Albert Einstein', 'context': 'the theory of relativity'}

Generated Biography:
Albert Einstein, born on March 14, 1879, in Ulm, Germany, is one of the most celebrated physicists in history, best known for his groundbreaking theory of relativity. As a child, Einstein showed a deep curiosity about the mysteries of the universe, which eventually led him to revolutionize our understanding of space, time, and energy.

Einstein's journey to fame began in 1905, a year often referred to as his "miracle year." During this time, he published several pivotal papers, including one that introduced the special theory of relativity. This theory fundamentally changed the way we perceive the fabric of the universe by establishing that the laws of physics are the same for all non-accelerating observers and that the speed of light is constant, regardless of the observer's motion. A key outcome of this theory was the iconic equation \(E=mc^2\), which demon

> **What to Take Away ?**

- The chain does not depend on where the input comes from.
- Each step has a clear responsibility.
- Data preparation is explicit and readable.

This separation becomes essential when building agents:
reasoning, tools, and memory all rely on the same idea of controlled data flow.

### Parallel Processing with RunnableParallel

Until now, all chains followed a linear path:
each component processed the output of the previous one.

Agentic systems often need a different structure.
They must look at the **same input from several angles at once**.

Typical examples include:
- producing a summary and an analysis in parallel,
- extracting facts while keeping the original context,
- generating multiple candidates before making a decision.

LCEL supports this pattern through **RunnableParallel**.
It allows one input to feed several branches at the same time.


To illustrate this idea, we will reuse a single language model,
but ask it to perform **different tasks in parallel**.

In [22]:
model = ChatOpenAI(model="gpt-4o")
output_parser = StrOutputParser()

Each branch of the pipeline is defined by its own prompt.
All of them will receive the **same input dictionary**,
but they will produce different kinds of outputs.


In [23]:
summary_prompt = ChatPromptTemplate.from_template(
    "Write a short summary about {topic}."
)

key_points_prompt = ChatPromptTemplate.from_template(
    "List three key points about {topic}."
)

These branches are then combined into a single parallel structure.
`RunnableParallel` takes care of sending the input to all branches
and collecting their outputs in a structured way.


In [26]:
parallel_chain = RunnableParallel(
    summary=summary_prompt | model | output_parser,
    key_points=key_points_prompt | model | output_parser,
)

When the chain is invoked, both branches are executed independently.
The result is a dictionary that groups all outputs under clear keys.

In [ ]:
input_data = {"topic": "Agentic AI"}
result = parallel_chain.invoke(input_data)

print("----- Parallel Processing Result -----")
print("\nSummary:")
print(result["summary"])

print("\nKey Points:")
print(result["key_points"])

----- Parallel Processing Result -----

Summary:
Agentic AI refers to a branch of artificial intelligence research and development focused on creating systems that exhibit autonomous decision-making and problem-solving capabilities. These AI systems are designed to operate with a degree of independence, making proactive decisions to achieve specific goals or tasks without requiring constant human oversight. Agentic AI can be applied in various domains, from robotics and personal assistants to autonomous vehicles and smart environments. The development of Agentic AI involves challenges such as ensuring ethical behavior, understanding complex environments, and interacting seamlessly with humans and other systems. The ultimate goal is to create AI agents that can adapt to new situations, learn from experiences, and make decisions that align with human values and preferences.

Key Points:
Agentic AI refers to artificial intelligence systems that exhibit characteristics typically associated

The data flow is now easy to reason about:

- one input enters the system,
- several transformations happen in parallel,
- results are gathered without conflict.

This structured parallelism is essential for agents.
Before acting, an agent may need to:
- analyze a situation,
- check constraints,
- generate possible actions.

`RunnableParallel` makes this pattern explicit and manageable.

## Integrating Your First LLM (OpenAI)

So far, our chains focused on **structure**:
how prompts, runnables, and parsers connect together.

At the center of all these pipelines sits one critical component:
the **model**.

The model is the part that performs the actual reasoning.
It reads the prompt, interprets the context, and generates a response.

### Why the Model Abstraction Matters

LangChain provides a **standard interface** to interact with many different language models:
- OpenAI
- Anthropic
- open-source models
- self-hosted models

This design choice has an important consequence:

> You are not locked into a single provider.

The rest of your chain (prompts, parsers, runnables) can stay the same
even if you change the underlying model later.

### Why Start with OpenAI?

Many developers begin with OpenAI models because they:
- perform well across many tasks,
- follow instructions reliably,
- integrate easily into existing workflows.

In LangChain, OpenAI chat models are accessed through the `ChatOpenAI` class.

To make things concrete, we will build a **minimal chain** that uses
`gpt-4o` as its reasoning engine.

The goal is not complexity,
but to clearly see how a model fits into the LCEL pipeline.

In [28]:
model = ChatOpenAI(model="gpt-4o", temperature=0.3)

The temperature controls how deterministic the output is:
- lower values → more stable, repeatable answers
- higher values → more creative, varied answers

For learning and debugging, lower values are usually better.

### Connecting the Model to a Prompt

The model itself does nothing without a prompt.
We start with a simple instruction that takes one input variable.

In [29]:
promp_template = ChatPromptTemplate.from_template("Explain the concept of {topic} in simple terms.")

The model response is a rich object.

To keep things simple, we will extract only the generated text.

In [30]:
output_parser = StrOutputParser()

With all components defined, we can now connect them into a single chain.

In [31]:
chain = promp_template | model | output_parser

When the chain is invoked, the input flows through the prompt, then into the model, and finally into the parser.

In [32]:
input_data = {"topic": "agentic AI"}

result = chain.invoke(input_data)

print("----- Model Output -----")
print(result)

----- Model Output -----
Agentic AI refers to artificial intelligence systems that can act autonomously and make decisions on their own to achieve specific goals. These systems are designed to perform tasks without needing constant human intervention. They can perceive their environment, process information, and take actions that help them accomplish their objectives. Essentially, agentic AI can be thought of as AI with a degree of independence, capable of adapting to new situations and making choices based on the data it receives and its programmed goals.


## Understanding Streaming, Batching, and Asynchronous Calls with LCEL

So far, we have used `invoke()` to run a chain and get a single result.
This works well for simple experiments and demos.

Real-world applications have different constraints:
- users expect **fast and continuous feedback**,
- systems must handle **many requests efficiently**,
- backends often need to **run tasks concurrently**.

LCEL supports these needs through three execution modes:
**streaming**, **batching**, and **asynchronous calls**.

Each mode solves a different problem, but all of them use the same chain structure.


We will reuse a simple chain to focus on *how* it is executed,
not on *what* it does.

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0.4)

prompt_template = ChatPromptTemplate.from_template(
    "Give a short explanation of {topic}."
)

output_parser = StrOutputParser()

chain = prompt_template | model | output_parser

### Streaming — Real-Time Feedback

In interactive applications (chat interfaces, assistants, dashboards),
waiting for the full response can feel slow.

**Streaming** solves this by returning partial outputs as soon as they are generated.

Instead of waiting for the final answer,
the user sees the response appear token by token.


In [40]:
input = {"topic": "Agentic AI"}
for chunk in chain.stream(input):
    print(chunk, end="", flush=True)

Agentic AI refers to artificial intelligence systems that are designed to operate with a degree of autonomy, making decisions and taking actions in pursuit of specific goals. These systems are often characterized by their ability to perceive their environment, reason about the information they gather, and make decisions based on predefined objectives or learned experiences. Agentic AI can be employed in various applications, such as autonomous vehicles, personal assistants, and robotics, where they act as agents that can interact with the world and adapt to changing conditions. The concept emphasizes the AI's capability to act independently, rather than merely following scripted instructions.

What happens here:
- the chain starts generating text,
- each chunk is sent immediately,
- the full response is built progressively.

Streaming improves **perceived performance**
and is especially useful for user-facing agentic systems.

### Batching — High-Throughput Processing

Sometimes the goal is not speed for one request, but efficiency across **many requests**.

Batching allows the same chain to process multiple inputs in one call.

In [43]:
inputs = [
    {"topic": "agentic AI"},
    {"topic": "retrieval-augmented generation"},
    {"topic": "LLM orchestration"},
]

results = chain.batch(inputs)

for topic, output in zip(inputs, results):
    print(f"\nTopic: {topic['topic']}")
    print(f"Output: {output}")


Topic: agentic AI
Output: Agentic AI refers to artificial intelligence systems designed to operate with a degree of autonomy, making decisions and taking actions to achieve specific goals without requiring constant human oversight. These systems are often equipped with the ability to perceive their environment, process information, and adapt to new situations, enabling them to perform tasks or solve problems in dynamic settings. The concept of agentic AI is closely related to the idea of intelligent agents, which are software entities that act on behalf of users to perform tasks such as information retrieval, decision-making, and automation. The development of agentic AI involves ensuring that these systems can operate safely and align with human values and intentions.

Topic: retrieval-augmented generation
Output: Retrieval-Augmented Generation (RAG) is a technique that combines the strengths of retrieval-based and generation-based models to improve the quality and accuracy of genera

Batching is useful when:
- processing large datasets,
- running offline jobs,
- evaluating prompts at scale.

From a conceptual point of view,
the chain stays the same — only the **execution mode changes**.

### Asynchronous Calls — Concurrent Execution

Modern applications often handle multiple tasks at the same time:
- several users,
- multiple agents,
- parallel tool calls.

LCEL supports asynchronous execution through `ainvoke()`.

In [45]:
result = await chain.ainvoke({"topic": "multi-agent systems"})
print(result)

Multi-agent systems (MAS) are computational systems in which multiple autonomous entities, known as agents, interact or work together to achieve individual or collective goals. Each agent in a MAS can perceive its environment, make decisions, and perform actions based on its own objectives and the information it gathers. These systems are often used to solve complex problems that are difficult or impossible for a single agent to tackle alone. Agents in a MAS can collaborate, compete, or coexist, and they may have varying degrees of independence and capabilities. Applications of multi-agent systems include robotics, distributed control systems, simulation, and artificial intelligence, where they can model and manage decentralized processes, enhance scalability, and improve robustness.


With asynchronous calls:
- the chain does not block the main program,
- multiple chains can run concurrently,
- resources are used more efficiently.

This mode is essential for scalable agent architectures.

## Your First "Tool" — Building a Simple Function and a Runnable

Up to now, our chains mostly produced *text*.
This is useful, but agentic systems need more than text.

Agents must be able to **do something** with model outputs:
- transform them,
- validate them,
- extract structured signals,
- call external systems (APIs, databases, search, etc.).

Before introducing real tool calling, we start with the simplest idea:

> a tool can be a plain Python function.

In LangChain, a function can become a chain component by wrapping it as a Runnable.
That is exactly what `RunnableLambda` is for.

In this section, the function will be the "tool":
it will take the LLM output and apply a deterministic transformation.

To keep the example concrete, we will build a small chain that:
1) asks the model to generate a short list,
2) converts that text into a Python list,
3) applies a function to post-process the result.

This mirrors a common agent pattern:
LLM output → parsing/cleanup → deterministic logic.

In [47]:
model = ChatOpenAI(model="gpt-4o", temperature=0.2)

prompt_template = ChatPromptTemplate.from_template(
    "Give {n} short action ideas for an AI agent helping with {topic}. "
    "Return them as a simple bullet list."
)

output_parser = StrOutputParser()

Now we define a small Python function.
This function acts like a “tool” because it is:
- deterministic,
- testable,
- independent from the model.

The goal here is simple: extract clean items from the bullet list.

In [48]:
def extract_bullets(text: str) -> list[str]:
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    items = []
    for line in lines:
        # remove common bullet markers
        cleaned = line.lstrip("-•*0123456789. ").strip()
        if cleaned:
            items.append(cleaned)
    return items

`RunnableLambda` turns this Python function into a runnable component.
This makes it composable with LCEL, just like prompts or models.

In [49]:
bullet_extractor = RunnableLambda(extract_bullets)

We can now build a complete pipeline:

- the prompt formats the input,
- the model generates text,
- the parser extracts a string,
- the function converts the string into a structured Python list.

In [50]:
chain = prompt_template | model | output_parser | bullet_extractor

When the chain runs, the final output is no longer raw text.
It becomes structured data that the rest of an agent system can use.

In [51]:
input_data = {"n": 5, "topic": "calendar scheduling"}
actions = chain.invoke(input_data)

print("----- Extracted Actions (Python list) -----")
for i, a in enumerate(actions, start=1):
    print(f"{i}. {a}")


----- Extracted Actions (Python list) -----
1. Automatically suggest optimal meeting times based on participants' availability and time zones.
2. Send reminders and follow-up notifications for upcoming meetings and deadlines.
3. Integrate with email to propose meeting times directly within email threads.
4. Analyze past scheduling patterns to recommend the best days and times for recurring meetings.
5. Automatically resolve scheduling conflicts by suggesting alternative times or notifying participants.


This example is not a tool in the LangChain “Tools API” sense yet.
But it captures the core idea:

- the LLM provides flexible generation,
- Python provides reliable execution.

Agentic systems live at the intersection of these two worlds.

A real agent will use tools to:
- search the web,
- query a database,
- call an API,
- trigger workflows.

The mental model remains the same:
LLM output → runnable logic → next decision.

This chapter introduced the foundations needed for agentic development:

- composability with LCEL,
- handling multiple inputs,
- parallel execution,
- different execution modes (stream, batch, async),
- turning Python logic into runnable components.

With these building blocks, we are ready to move from *chains*
to real agent workflows.

In the next chapter, We will use these same composable pieces,
but we will organize them inside a graph-based loop:
observe → think → act → observe again.
